In [ ]:
import eodslib
from datetime import datetime
from pathlib import Path
from osgeo import gdal

In [ ]:
# user connection settings
conn = {
    'domain': 'https://earthobs.defra.gov.uk',
    'username': '<insert-user-name>',
    'access_token': '<insert-user-token>',
    }

In [ ]:
# user defined keyword arguments (kwargs)
output_dir = Path('/some/dir')
cat_search_geom_wgs84wkt_query = 'POLYGON((-1.64 53.38, -1.64 52.98, -1.17 52.98, -1.17 53.38, -1.64 53.38))'

In [ ]:
"""
You can also use the config.py to import these settings and apply them across any notebook
See below (commented out) example on how to import settings
"""
# import config
# conn = config.conn
# output_dir = config.output_dir

In [ ]:
# example of EODS query for generating a cloud free mosaic for Summer 2020
# since no 'geom' parameter is set, this will query every granule reference with the catalogue

eods_params = {
    'output_dir':output_dir,
    'sat_id':2,
    'date_start':'2020-05-01',
    'date_end':'2020-06-30',
    'find_least_cloud':True,
    'ignore_split_granules':True,
    }

list_of_layers, df = eodslib.query_catalog(conn, **eods_params)

In [ ]:
# this is an example where you want to override the list of layers returned by the EODS query cell above
# list_of_layers = ['geonode:S2B_20200424_lat52lon22_T30UWC_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref',
#                   'geonode:S2B_20200424_lat53lon219_T30UWD_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref'
#                  ]

In [ ]:
# create an empty list of results
list_of_results = list()

# iterative through the returned EODS query list of layers
for lyr in list_of_layers:

    # for each layer, create a wps config dictionary, inserting the layer name
    config_wpsprocess = {'template_xml':'gsdownload_template.xml',
        'xml_config':{
            'template_layer_name':lyr,
            'template_outputformat':'image/tiff',
            'template_mimetype':'application/zip'
                },
        'dl_bool':True
    }

    # for each layer, call the run wps function which:
        # submits the WPS job
        # polls the wps server until the job is READY
        # downloads the result (and retries up to 3 times if there's a break in the download connection)
    execution_dict = eodslib.run_wps(conn, config_wpsprocess, output_dir=output_dir)

    # append the execution dictionary to the results list
    list_of_results.append(execution_dict)

# log the output to a log csv file
eodslib.output_log(list_of_results)

In [ ]:
# POST-PROCESSING EXAMPLE
#        convert gsdownload s2 output to an "optimised" GeoTIFF
#        build a gdal virtual file (VRT) (a mosaic)

# set gdal config
translateoptions = gdal.TranslateOptions(gdal.ParseCommandLine("-of Gtiff -co COMPRESS=LZW  -co BIGTIFF=YES -co TILED=YES -co BLOCKXSIZE=512 -co BLOCKYSIZE=512"))
gdal.SetConfigOption("COMPRESS_OVERVIEW", "DEFLATE")

outfile_list = []

# loop through all S2 output in output directory
for input_file in output_dir.glob('**/*osgb_vmsk_sharp_rad_srefdem_stdsref.tif*'):
    
    print('\n' + datetime.utcnow().isoformat() + ' processing input :: ' + str(input_file))
    
    output_file = input_file.parent / str(input_file.stem + '-cog.tif')
    outfile_list.append(str(output_file))
    
    # run downloaded file via gdal translate
    gdal.Translate(str(output_file), str(input_file), options=translateoptions)
    
    # add overviews to translated file
    Image = gdal.Open(str(output_file), 1)    
    Image.BuildOverviews("NEAREST", [2,4,8,16,32,64, 128, 256, 512])
    
    print('\n' + datetime.utcnow().isoformat() + ' output here :: ' + str(output_file))
    
# write out vrt input file    
with open(output_dir / 'vrt-inputs.txt', 'w') as outtxt:
    for ii in outfile_list:
        outtxt.write(ii + '\n')

# run gdal build vrt from commandline
! gdalbuildvrt -resolution average -r nearest -input_file_list {str(output_dir / 'vrt-inputs.txt')} {str(output_dir / 'mosaic.vrt')}

In [ ]:
print('### Script Finished')